In [ ]:
import requests, os

HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY")

url = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-3-medium-diffusers"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

payload = {
    "inputs": "A beautiful sunset",
    "parameters": {
        "width": 1024,
        "height": 1024,
        "num_inference_steps": 20,
        "guidance_scale": 7.5,
        "negative_prompt": "blurry, low quality, distorted, ugly, bad anatomy"
    }
}

res = requests.post(url, headers=headers, json=payload)

if res.status_code == 200:
    with open("sunset.png", "wb") as f:
        f.write(res.content)
    print("Image saved as sunset.png")
else:
    print(f"Error {res.status_code}: {res.text}")


Image saved as sunset.png


# Starting

In [24]:

from typing import Dict, Any, List

def generate_static_images(
    content_strategy: Dict[str, Any],
    brand_colors: str = "",
    include_product_visuals: bool = False
) -> List[Dict[str, Any]]:
    """
    Generate static images for marketing campaigns
    
    Args:
        content_strategy: Content strategy and requirements
        brand_colors: Brand color scheme
        include_product_visuals: Whether to include product-specific visuals
    
    Returns:
        List of generated images with metadata
    """
    
    images = []
    
    # Generate different types of static images
    image_types = [
        {"type": "hero_image", "description": "Main campaign hero image"},
        {"type": "social_post", "description": "Social media post image"},
        {"type": "ad_creative", "description": "Advertising creative"}
    ]
    
    if include_product_visuals:
        image_types.append({"type": "product_showcase", "description": "Product showcase image"})
    
    for image_type in image_types:
        try:
            image = generate_single_static_image(
                content_strategy=content_strategy,
                image_type=image_type["type"],  
                brand_colors=brand_colors,
                description=image_type["description"]
            )
            images.append(image)
        except Exception as e:
            # Create error placeholder
            images.append({
                "type": image_type["type"],
                "description": image_type["description"],
                "error": str(e),
                "status": "failed"
            })
    
    return images

In [25]:
def generate_single_static_image(
    content_strategy: Dict[str, Any],
    image_type: str,
    brand_colors: str,
    description: str
) -> Dict[str, Any]:
    """Generate a single static image"""
    
    # Create image generation prompt
    image_prompt = create_image_prompt(
        content_strategy=content_strategy,
        image_type=image_type,
        brand_colors=brand_colors,
        description=description
    )
    
    # Generate image using the image generation tool
    image_result = generate_image(
        prompt=image_prompt,
        style=get_image_style(image_type),
        dimensions=get_image_dimensions(image_type)
    )
    
    # Structure the image data
    image = {
        "type": image_type,
        "description": description,
        "prompt": image_prompt,
        "url": image_result.get("url", ""),
        "dimensions": get_image_dimensions(image_type),
        "style": get_image_style(image_type),
        "brand_colors": brand_colors,
        "usage_rights": "Generated for campaign use",
        "alt_text": generate_alt_text(image_type, content_strategy),
        "file_name": f"{image_type}_{generate_timestamp()}.png",
        "variations": generate_image_variations(image_type, image_prompt),
        "optimization": get_optimization_settings(image_type)
    }
    return image